In [ ]:
#!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
#!pip install tavily-python crewai-tools

In [4]:
import warnings
warnings.filterwarnings('ignore')
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')

from crewai import Agent, Task, Crew


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [9]:
# Agents
planner = Agent(
    role="High-Level Trip Planner",
    goal="Decompose the vacation goal into a hierarchy of sub-goals and prioritized tasks.",
    backstory="You break down large travel goals into clear, ordered subtasks and define success criteria.",
    llm=llm,
    verbose=True
)

researcher = Agent(
    role="Travel Researcher",
    goal="Find practical options for flights, accommodations, and local transport given constraints.",
    backstory="You are skilled at quickly comparing travel options and returning concise recommendations.",
    llm=llm,
    verbose=True
)

budgeter = Agent(
    role="Travel Budget Analyst",
    goal="Estimate costs across flights, hotels, transport, food, and activities and recommend a budget.",
    backstory="You create realistic cost estimates and suggest cost-saving tradeoffs for families.",
    llm=llm,
    verbose=True
)

itinerary_maker = Agent(
    role="Itinerary Creator",
    goal="Produce a detailed day-by-day itinerary aligned with family preferences, pace, and child-friendly options.",
    backstory="You craft balanced itineraries with travel time, rest, and high-impact experiences.",
    llm=llm,
    verbose=True
)

coordinator = Agent(
    role="Trip Coordinator",
    goal="Assemble outputs from other agents into a single cohesive travel plan and checklist.",
    backstory="You synthesize plans into an actionable final deliverable and list next steps for booking.",
    llm=llm,
    verbose=True
)


# Tasks (use dynamic placeholders)


decompose_task = Task(
    description=(
        "Decompose the high-level goal:\n"
        "Goal: {vacation_goal}\n"
        "Dates: {dates}\n"
        "Family size: {family_size}\n"
        "Preferences: {preferences}\n\n"
        "Produce a hierarchical plan with levels:\n"
        "Level 1: Major pillars (travel, lodging, transport, activities, budget)\n"
        "Level 2: For each pillar, list 3-5 concrete sub-tasks\n"
        "Level 3: For the top-priority subtasks, suggest success criteria and timelines.\n\n"
        "Return as a structured list or JSON-like text."
    ),
    expected_output="Hierarchical plan (pillars -> subtasks -> success criteria)",
    agent=planner
)

research_task = Task(
    description=(
        "Research travel options given:\n"
        "Goal: {vacation_goal}\n"
        "Dates: {dates}\n"
        "Family size: {family_size}\n"
        "Preferences: {preferences}\n\n"
        "Return concise recommendations for:\n"
        "- 2 flight options (price tier and rationale)\n"
        "- 2 accommodation types (hotel vs apartment) and one sample neighborhood each\n"
        "- Local transport options (rail passes, car rental notes)\n\n"
        "If any option requires online bookings or external checks, flag it with 'ACTION REQUIRED'."
    ),
    expected_output="Research summary with 2-3 recommended options per category",
    agent=researcher
)

budget_task = Task(
    description=(
        "Create a budget estimate for the trip described:\n"
        "Goal: {vacation_goal}\n"
        "Dates: {dates}\n"
        "Family size: {family_size}\n"
        "Preferences: {preferences}\n\n"
        "Include estimated costs (per category): Flights, Accommodation (per night), Local transport, Food (per day), Activities, Contingency.\n"
        "Give a 'Total estimated budget' and two cost-optimization suggestions."
    ),
    expected_output="Budget table and cost-saving recommendations",
    agent=budgeter
)

itinerary_task = Task(
    description=(
        "Build a 7-day sample itinerary (adjust to {dates} length if needed) for the family:\n"
        "Goal: {vacation_goal}\n"
        "Family size: {family_size}\n"
        "Preferences: {preferences}\n\n"
        "For each day list:\n"
        "- Morning / Afternoon / Evening items\n"
        "- Estimated travel time between stops\n"
        "- Child-friendly notes and booking needs\n"
        "- Time buffers for rest\n\n"
        "Return a balanced, realistic daily schedule."
    ),
    expected_output="Day-by-day itinerary with notes",
    agent=itinerary_maker
)

coordinate_task = Task(
    description=(
        "Assemble final travel plan using previous outputs.\n"
        "Inputs: hierarchical plan, research summary, budget estimate, itinerary.\n\n"
        "Produce a final deliverable containing:\n"
        "- Executive summary (1-2 paragraphs)\n"
        "- Actionable booking checklist (flights, hotels, passes, reservations)\n"
        "- Timeline for when to book each item\n"
        "- Risk notes and contingencies\n"
        "- Final total estimated cost\n\n"
        "Format clearly for sharing with family."
    ),
    expected_output="Final trip plan & checklist",
    agent=coordinator
)

# Build Crew

vacation_crew = Crew(
    llm=llm,
    agents=[planner, researcher, budgeter, itinerary_maker, coordinator],
    tasks=[decompose_task, research_task, budget_task, itinerary_task, coordinate_task],
    verbose=2
)



In [10]:
vacation_goal = "Plan a 7-day family vacation to Italy focusing on Rome, Florence, and Venice."
dates = "2025-06-10 to 2025-06-17"
family_size = "2 adults, 2 children (ages 8 and 5)"
preferences = "Moderate budget, kid-friendly activities, avoid long overnight travel, prefer central accommodations"


In [11]:
inputs = {
        "vacation_goal": vacation_goal,
        "dates": dates,
        "family_size": family_size,
        "preferences": preferences
    }

result = vacation_crew.kickoff(inputs)

 [DEBUG]: == Working Agent: High-Level Trip Planner
 [INFO]: == Starting Task: Decompose the high-level goal:
Goal: Plan a 7-day family vacation to Italy focusing on Rome, Florence, and Venice.
Dates: 2025-06-10 to 2025-06-17
Family size: 2 adults, 2 children (ages 8 and 5)
Preferences: Moderate budget, kid-friendly activities, avoid long overnight travel, prefer central accommodations

Produce a hierarchical plan with levels:
Level 1: Major pillars (travel, lodging, transport, activities, budget)
Level 2: For each pillar, list 3-5 concrete sub-tasks
Level 3: For the top-priority subtasks, suggest success criteria and timelines.

Return as a structured list or JSON-like text.


> Entering new CrewAgentExecutor chain...
Thought: I need to break down the vacation goal into a structured hierarchical plan that includes major pillars, subtasks, and success criteria. I will start by identifying the major pillars of the trip.

Action: Delegate work to co-worker
Action Input: {
    "coworker":

In [12]:
print(result)

### Executive Summary
This travel plan outlines a 7-day family trip to Italy from June 10 to June 17, 2025, for two adults and two children (ages 8 and 5). The itinerary includes visits to Rome, Florence, and Venice, focusing on kid-friendly activities and central accommodations. Each day is balanced with sightseeing, meals, and rest periods to ensure an enjoyable experience for the whole family. The estimated total cost for the trip is approximately $4,975, allowing for a comfortable budget while exploring Italy's rich culture and history.

### Actionable Booking Checklist
1. **Flights:**
   - Book round-trip airfare for 2 adults and 2 children.
   
2. **Accommodations:**
   - Reserve hotels:
     - Rome: Hotel Artemide or Hotel Quirinale
     - Florence: Hotel Brunelleschi or Hotel L'Orologio
     - Venice: Hotel Antiche Figure or Hotel Canal Grande

3. **Train Tickets:**
   - Book high-speed train tickets:
     - Rome to Florence
     - Florence to Venice

4. **Attraction Tickets:**

In [17]:
load_dotenv()

True

In [18]:
from crewai_tools import SerperDevTool
search_tool = SerperDevTool()

In [ ]:
## Test SERPER
from crewai_tools import SerperDevTool
# Initialize tool
search_tool = SerperDevTool()
# Simple test query
result = search_tool.run("Flight option for Rome")
print(result)

In [20]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2
)

# ---- Tool ----
search_tool = SerperDevTool()

# ---- Agents ----
planner = Agent(
    role="High-Level Trip Planner",
    goal="Break the vacation goal into a clear hierarchy of tasks.",
    backstory="Expert at decomposing complex travel goals.",
    llm=llm,
    verbose=True
)

researcher = Agent(
    role="Travel Researcher",
    goal="Find flight options and transport info using live web search.",
    backstory="Compares real-world travel options quickly.",
    llm=llm,
    tools=[search_tool],
    verbose=True
)

itinerary_maker = Agent(
    role="Itinerary Creator",
    goal="Create a realistic itinerary using real attractions.",
    backstory="Builds practical, family-friendly travel plans.",
    llm=llm,
    tools=[search_tool],
    verbose=True
)

coordinator = Agent(
    role="Trip Coordinator",
    goal="Combine all outputs into a final travel plan.",
    backstory="Synthesizes plans into a clear final checklist.",
    llm=llm,
    verbose=True
)

# ---- Tasks (EXPECTED_OUTPUT ADDED) ----
decompose_task = Task(
    description=(
        "Decompose the trip goal:\n"
        "Goal: Family vacation to Italy\n"
        "Dates: June 2025, 7 days\n"
        "Family size: 4\n"
        "Preferences: culture, food, relaxed pace\n\n"
        "Return major pillars and key subtasks."
    ),
    expected_output="Hierarchical list of major pillars and subtasks",
    agent=planner
)

research_task = Task(
    description=(
        "Using live web search, find:\n"
        "- 2 economy flight options to Italy (airline + price range)\n"
        "- 1 premium flight option\n"
        "- Key local transport options\n\n"
        "Base results on recent search data."
    ),
    expected_output="Flight options and transport recommendations with rationale",
    agent=researcher
)

itinerary_task = Task(
    description=(
        "Using live web search, create a 5-day family-friendly itinerary for Italy.\n"
        "Include:\n"
        "- Major attractions\n"
        "- Morning/Afternoon/Evening split\n"
        "- Booking requirements\n"
        "- Avoid over-packed days"
    ),
    expected_output="Day-by-day itinerary grounded in real attractions",
    agent=itinerary_maker
)

coordinate_task = Task(
    description=(
        "Combine all previous outputs into a final plan with:\n"
        "- Executive summary\n"
        "- Booking checklist\n"
        "- Key risks\n"
        "- Next steps"
    ),
    expected_output="Final consolidated travel plan and checklist",
    agent=coordinator
)

# ---- Crew ----
crew = Crew(
    agents=[planner, researcher, itinerary_maker, coordinator],
    tasks=[decompose_task, research_task, itinerary_task, coordinate_task],
    verbose=2
)

# ---- Run ----
result = crew.kickoff()


2026-01-17 22:49:44,864 - 8615996992 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: High-Level Trip Planner
 [INFO]: == Starting Task: Decompose the trip goal:
Goal: Family vacation to Italy
Dates: June 2025, 7 days
Family size: 4
Preferences: culture, food, relaxed pace

Return major pillars and key subtasks.


> Entering new CrewAgentExecutor chain...
I need to break down the family vacation goal into a clear hierarchy of tasks. This involves identifying major pillars related to the trip and then detailing the key subtasks under each pillar. 

Action: Delegate work to co-worker  
Action Input: { "coworker": "Travel Researcher", "task": "Research cultural attractions, food experiences, and relaxed pace activities in Italy for a family vacation.", "context": "The family vacation is planned for June 2025 and will last for 7 days. There are 4 family members, and they have a preference for culture, food, and a relaxed pace. The research should focus on family-friendly activities that align with these preferences." }  

> Entering new CrewAgent

In [21]:
print("\n\n===== FINAL OUTPUT =====\n")
print(result)



===== FINAL OUTPUT =====

### Final Travel Plan for a 5-Day Family-Friendly Trip to Italy: Venice, Florence, and Rome

#### Executive Summary
This 5-day family-friendly itinerary explores the enchanting cities of Venice, Florence, and Rome, offering a blend of cultural experiences, historical landmarks, and fun activities suitable for all ages. Families will enjoy a mix of guided tours, leisure time, and interactive experiences that cater to both adults and children.

**Day 1: Arrival in Venice**
- **Key Attractions:** St. Mark's Basilica, Doge's Palace, and a gondola ride through the canals.
- **Accommodation:** Family-friendly hotels such as Hotel Antiche Figure or Hotel Canal Grande.
- **Dining:** Dinner at Trattoria Al Gatto Nero.

**Day 2: Venice**
- **Key Attractions:** Doge's Palace, Bridge of Sighs, and Murano Island.
- **Accommodation:** Same as Day 1.
- **Dining:** Lunch at a local café and dinner at Osteria alle Testiere.

**Day 3: Venice to Florence**
- **Travel Logistics

## Custom Tool

In [22]:
# ===============================
# 1️⃣ Imports
# ===============================
from crewai import Agent, Task, Crew
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from tavily import TavilyClient


# ===============================
# 2️⃣ LLM
# ===============================
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2
)


# ===============================
# 3️⃣ Custom Tavily Tool
# ===============================
tavily_client = TavilyClient(api_key="YOUR_TAVILY_API_KEY")


@tool
def italy_attraction_search(query: str) -> str:
    """
    Custom tool to fetch real Italy attractions using Tavily
    """
    response = tavily_client.search(
        query=query,
        search_depth="advanced",
        max_results=5
    )

    results = []
    for item in response.get("results", []):
        results.append(
            f"- {item.get('title')} ({item.get('url')})"
        )

    return "\n".join(results)


# ===============================
# 4️⃣ Agents
# ===============================
planner = Agent(
    role="High-Level Trip Planner",
    goal="Break the vacation goal into a clear hierarchy of tasks.",
    backstory="Expert at decomposing complex travel goals.",
    llm=llm,
    verbose=True
)

researcher = Agent(
    role="Travel Researcher",
    goal="Find flight options and transport info.",
    backstory="Compares real-world travel options quickly.",
    llm=llm,
    verbose=True
)

itinerary_maker = Agent(
    role="Itinerary Creator",
    goal="Create a realistic itinerary using real attractions.",
    backstory="Builds practical, family-friendly travel plans.",
    llm=llm,
    tools=[italy_attraction_search],  # ✅ Custom Tool Used Here
    verbose=True
)

coordinator = Agent(
    role="Trip Coordinator",
    goal="Combine all outputs into a final travel plan.",
    backstory="Synthesizes plans into a clear final checklist.",
    llm=llm,
    verbose=True
)


# ===============================
# 5️⃣ Tasks
# ===============================
decompose_task = Task(
    description=(
        "Decompose the trip goal:\n"
        "Goal: Family vacation to Italy\n"
        "Dates: June 2025, 7 days\n"
        "Family size: 4\n"
        "Preferences: culture, food, relaxed pace\n\n"
        "Return major pillars and key subtasks."
    ),
    expected_output="Hierarchical list of major pillars and subtasks",
    agent=planner
)

research_task = Task(
    description=(
        "Find:\n"
        "- 2 economy flight options to Italy\n"
        "- 1 premium flight option\n"
        "- Key local transport options"
    ),
    expected_output="Flight options and transport recommendations",
    agent=researcher
)

itinerary_task = Task(
    description=(
        "Using live search, create a 5-day family-friendly itinerary for Italy.\n"
        "Include:\n"
        "- Major attractions\n"
        "- Morning/Afternoon/Evening split\n"
        "- Booking requirements\n"
        "- Avoid over-packed days"
    ),
    expected_output="Day-by-day itinerary grounded in real attractions",
    agent=itinerary_maker
)

coordinate_task = Task(
    description=(
        "Combine all outputs into a final plan with:\n"
        "- Executive summary\n"
        "- Booking checklist\n"
        "- Key risks\n"
        "- Next steps"
    ),
    expected_output="Final consolidated travel plan",
    agent=coordinator
)


# ===============================
# 6️⃣ Crew & Run
# ===============================
crew = Crew(
    agents=[planner, researcher, itinerary_maker, coordinator],
    tasks=[decompose_task, research_task, itinerary_task, coordinate_task],
    verbose=2
)

result = crew.kickoff()



2026-01-17 23:18:12,459 - 8615996992 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


 [DEBUG]: == Working Agent: High-Level Trip Planner
 [INFO]: == Starting Task: Decompose the trip goal:
Goal: Family vacation to Italy
Dates: June 2025, 7 days
Family size: 4
Preferences: culture, food, relaxed pace

Return major pillars and key subtasks.


> Entering new CrewAgentExecutor chain...
I need to break down the family vacation goal into a clear hierarchy of tasks, focusing on major pillars and key subtasks. To do this effectively, I will first gather information on the various aspects of planning a trip to Italy, including cultural experiences, food options, and a relaxed itinerary suitable for a family of four.

Action: Delegate work to co-worker  
Action Input: { "coworker": "Travel Researcher", "task": "Research cultural attractions, food experiences, and relaxed activities in Italy for a family vacation.", "context": "The family vacation is planned for June 2025, lasting 7 days, and involves 4 family members. The family prefers cultural experiences, food-related activit

2026-01-17 23:18:43,085 - 13084114944 - __init__.py-__init__:192 - ERROR: Exception while exporting Span.
Traceback (most recent call last):
  File "/opt/anaconda3/envs/crewenv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/crewenv/lib/python3.12/site-packages/urllib3/connection.py", line 571, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/crewenv/lib/python3.12/http/client.py", line 1430, in getresponse
    response.begin()
  File "/opt/anaconda3/envs/crewenv/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/crewenv/lib/python3.12/http/client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
          

I now can give a great answer  
Final Answer: Here are two economy flight options and one premium flight option to Italy, along with key local transport options available in the country:

### Flight Options

#### Economy Flight Option 1:
- **Airline:** Delta Airlines
- **Departure City:** New York City (JFK)
- **Destination:** Rome (FCO)
- **Price:** $550 (round trip)
- **Departure Date:** March 15, 2024
- **Return Date:** March 22, 2024
- **Flight Duration:** Approximately 8 hours direct

#### Economy Flight Option 2:
- **Airline:** United Airlines
- **Departure City:** Chicago (ORD)
- **Destination:** Milan (MXP)
- **Price:** $600 (round trip)
- **Departure Date:** March 10, 2024
- **Return Date:** March 17, 2024
- **Flight Duration:** Approximately 9 hours direct

#### Premium Flight Option:
- **Airline:** Lufthansa
- **Departure City:** Los Angeles (LAX)
- **Destination:** Florence (FLR) (via Munich)
- **Price:** $1,800 (round trip)
- **Departure Date:** March 12, 2024
- **Return D

In [ ]:
print("\n\n===== FINAL OUTPUT =====\n")
print(result)

In [23]:

# ---------- IMPORTS ----------
import os
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from tavily import TavilyClient

# ---------- API KEYS ----------
# Make sure these are set BEFORE running
# os.environ["OPENAI_API_KEY"] = "your_openai_key"
# os.environ["TAVILY_API_KEY"] = "your_tavily_key"

# ---------- LLM ----------
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.3
)

# =========================================================
# CUSTOM TOOL (TAVILY) — TOOL = CAPABILITY
# =========================================================

tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

@tool
def tavily_itinerary_search(query: str) -> str:
    """
    Search for real attractions and activities for itinerary planning.
    """
    response = tavily_client.search(
        query=query,
        search_depth="basic",
        max_results=5
    )

    results = []
    for r in response.get("results", []):
        results.append(f"- {r['title']}: {r['content']}")
    return "\n".join(results)

# =========================================================
# AGENTS
# =========================================================

planner = Agent(
    role="High-Level Trip Planner",
    goal="Break the vacation goal into structured travel pillars and priorities.",
    backstory="You think strategically and define what needs to be done.",
    llm=llm,
    verbose=True
)

itinerary_maker = Agent(
    role="Itinerary Creator",
    goal="Create a realistic, family-friendly itinerary using real attractions.",
    backstory="You design balanced travel days grounded in reality.",
    llm=llm,
    tools=[tavily_itinerary_search],
    verbose=True
)

critic = Agent(
    role="Itinerary Critic",
    goal="Evaluate itinerary quality and decide whether it should be revised.",
    backstory=(
        "You act as a reviewer who checks realism, pace, "
        "family-friendliness, and feasibility."
    ),
    llm=llm,
    verbose=True
)

coordinator = Agent(
    role="Trip Coordinator",
    goal="Finalize the approved itinerary into a polished travel plan.",
    backstory="You prepare the final shareable version.",
    llm=llm,
    verbose=True
)

# =========================================================
# TASKS
# =========================================================

decompose_task = Task(
    description=(
        "Decompose the goal:\n"
        "Family vacation to Italy, June 2025, 7 days, family of 4.\n"
        "Preferences: culture, food, relaxed pace.\n\n"
        "Return major planning pillars and key subtasks."
    ),
    expected_output="Hierarchical list of travel pillars and subtasks",
    agent=planner
)

itinerary_task = Task(
    description=(
        "Using live search results, create a 5-day family-friendly itinerary for Italy.\n\n"
        "For each day include:\n"
        "- Morning / Afternoon / Evening activities\n"
        "- Real attractions or neighborhoods\n"
        "- Travel time notes\n"
        "- Child-friendly considerations\n"
        "- Booking requirements\n\n"
        "Avoid rushing and include rest buffers."
    ),
    expected_output="Day-by-day itinerary grounded in real attractions",
    agent=itinerary_maker
)

evaluate_task = Task(
    description=(
        "Evaluate the itinerary on:\n"
        "1. Realistic pacing\n"
        "2. Family-friendliness\n"
        "3. Feasibility\n"
        "4. Balance of activities\n\n"
        "Give:\n"
        "- Score (1–10)\n"
        "- Improvement feedback\n"
        "- Final verdict: APPROVE or REVISE"
    ),
    expected_output="Score, feedback, and APPROVE/REVISE verdict",
    agent=critic
)

finalize_task = Task(
    description=(
        "Polish the approved itinerary into a final travel plan with:\n"
        "- Executive summary\n"
        "- Daily itinerary\n"
        "- Booking checklist\n"
        "- Risk notes"
    ),
    expected_output="Final polished travel plan",
    agent=coordinator
)

# =========================================================
# REINFORCEMENT LOOP CONTROLLER
# =========================================================

MAX_ITERATIONS = 2
approved = False
iteration = 0
itinerary_output = None

while not approved and iteration < MAX_ITERATIONS:
    iteration += 1
    print(f"\n================ ITERATION {iteration} =================\n")

    # Step 1: Generate itinerary
    itinerary_output = itinerary_task.execute()
    print("\n--- GENERATED ITINERARY ---\n")
    print(itinerary_output)

    # Step 2: Evaluate itinerary (reward signal)
    evaluation_output = evaluate_task.execute(
        context={"itinerary": itinerary_output}
    )
    print("\n--- CRITIC FEEDBACK ---\n")
    print(evaluation_output)

    # Step 3: Reinforcement decision
    if "APPROVE" in evaluation_output.upper():
        approved = True
        print("\n✅ Itinerary APPROVED")
    else:
        print("\n🔁 Itinerary needs improvement — reinforcing...")
        itinerary_task.description += (
            "\n\nReviewer feedback to address:\n"
            f"{evaluation_output}\n"
            "Revise the itinerary accordingly."
        )

# =========================================================
# FINALIZATION
# =========================================================

final_plan = finalize_task.execute(
    context={"itinerary": itinerary_output}
)





================ ITERATION 1 =================



> Entering new CrewAgentExecutor chain...
Action: tavily_itinerary_search  
Action Input: {"query": "family-friendly attractions in Italy"}   

- The best family destinations in Italy by age: ideas you'll ...: Rome, Florence and Venice, which are super exciting destinations in terms of variety of things to see, food to try, life to observe but also
- Italy Tours for Families: Top 5 Things to do on Italy Tours for Families · Visit Pompeii · Walk Along the Rim of Mt Vesuvius · Kayak on the Amalfi Coast · Go Swimming at Positano · Learn How
- 40 FUN & MEMORABLE THINGS TO DO IN ITALY WITH ...: From castle visits to tower climbs, pasta-making to bamboo labyrinths – there's something on this list for every Mario, Sofia, and Gianni in your family.
- THE 10 BEST Things to Do in Italy with Kids (2026): Things to Do in Italy with Kids · 1. Colosseum · 2. Pantheon · 3. Duomo di Milano · 4. Trevi Fountain · 5. Piazzale Michelangelo · 6. St.
- 15 B

2026-01-18 00:45:36,794 - 13084114944 - __init__.py-__init__:192 - ERROR: Exception while exporting Span.
Traceback (most recent call last):
  File "/opt/anaconda3/envs/crewenv/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/crewenv/lib/python3.12/site-packages/urllib3/connection.py", line 571, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/crewenv/lib/python3.12/http/client.py", line 1430, in getresponse
    response.begin()
  File "/opt/anaconda3/envs/crewenv/lib/python3.12/http/client.py", line 331, in begin
    version, status, reason = self._read_status()
                              ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/crewenv/lib/python3.12/http/client.py", line 292, in _read_status
    line = str(self.fp.readline(_MAXLINE + 1), "iso-8859-1")
          

Thought: I need to gather detailed information about family-friendly activities in Rome, Florence, and Venice to create a well-rounded 5-day itinerary that includes realistic pacing, child-friendly options, and a mix of cultural experiences.

Action: tavily_itinerary_search  
Action Input: {"query": "family-friendly activities Rome Florence Venice itinerary"} 
 

- Unforgettable Adventures from Venice to Rome - Kiss From Italy: This carefully designed one-week itinerary highlights the best family-friendly activities in Venice, Florence, and Rome, plus unique stops in
- Italy With Kids: The 5-Star Trip to Rome, Florence, & Venice: # Italy With Kids: Plan The Ultimate Rome, Florence, Venice Family Vacation. When planning a trip to Italy with kids the first time, I think it is easier to focus on the cities and towns with many planned family tours. The St. Regis Rome was the true “hotel star” of our family trip to Italy. Rome was the first European city to site a St. Regis Hotel, and certa

In [24]:
print("\n================ FINAL OUTPUT =================\n")
print(final_plan)


================ FINAL OUTPUT =================

### Final Travel Plan: 5-Day Family-Friendly Itinerary for Italy

#### Executive Summary
This travel plan outlines a 5-day family-friendly itinerary through Italy, focusing on Rome, Florence, Pisa, and Venice. Each day is designed to balance cultural exploration with child-friendly activities, ensuring an enjoyable experience for the whole family. The itinerary includes key attractions, travel times, dining options, and considerations for young travelers. A booking checklist and risk notes are also provided to facilitate a smooth journey.

---

#### Daily Itinerary

**#### Day 1: Arrival in Rome**  
- **Morning**: Arrive in Rome. Check into your family-friendly hotel (consider Hotel Artemide or similar). Rest and freshen up.  
- **Afternoon**: Visit the **Colosseum** (book tickets in advance to avoid long lines). Explore the ancient ruins and consider a guided tour that caters to families.  
  - **Travel Time**: 30 minutes from hotel (d